## Install TensorTrade

In [10]:
# !python3 -m pip install git+https://github.com/tensortrade-org/tensortrade.git

# !pip install stable_baselines
!pip install tensorforce

     |████████████████████████████████| 300 kB 7.7 MB/s 
     |████████████████████████████████| 75 kB 5.6 MB/s 


## Setup Data Fetching

In [1]:
import pandas as pd
import tensortrade.env.default as default

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC, ETH
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.agents import DQNAgent


%matplotlib inline

In [2]:
import pandas as pd
def load_csv(filename):
    df = pd.read_csv('data/' + filename, skiprows=0)
#     df.drop(columns=['symbol', 'volume_btc'], inplace=True)

    # Fix timestamp form "2019-10-17 09-AM" to "2019-10-17 09-00-00 AM"
#     df['date'] = df['date'].str[:14] + '00-00 ' + df['date'].str[-2:]

    # Convert the date column type from string to datetime for proper sorting.
    df['date'] = pd.to_datetime(df['date'])

    # Make sure historical prices are sorted chronologically, oldest first.
    df.sort_values(by='date', ascending=True, inplace=True)

    df.reset_index(drop=True, inplace=True)

    # Format timestamps as you want them to appear on the chart buy/sell marks.
    df['date'] = df['date'].dt.strftime('%Y-%m-%d %I:%M %p')

    return df
file_name = 'USD_CAD_H4_2021-05-07.csv'
data = load_csv(file_name)
data.describe()

,open,high,low,close,volume
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,1.318374,1.319967,1.316767,1.318357,7230.880200
std,0.036112,0.036399,0.035814,0.036127,7389.247272
min,1.215230,1.215970,1.212690,1.213680,1.000000
25%,1.299422,1.300977,1.297720,1.299392,2210.750000
50%,1.318335,1.319745,1.316970,1.318290,4609.000000
75%,1.333775,1.334937,1.332330,1.333712,9545.000000
max,1.462440,1.467060,1.454140,1.462470,59501.000000


## Create features with the feed module

In [3]:
def rsi(price: Stream[float], period: float) -> Stream[float]:
    r = price.diff()
    upside = r.clamp_min(0).abs()
    downside = r.clamp_max(0).abs()
    rs = upside.ewm(alpha=1 / period).mean() / downside.ewm(alpha=1 / period).mean()
    return 100*(1 - (1 + rs) ** -1)


def macd(price: Stream[float], fast: float, slow: float, signal: float) -> Stream[float]:
    fm = price.ewm(span=fast, adjust=False).mean()
    sm = price.ewm(span=slow, adjust=False).mean()
    md = fm - sm
    signal = md - md.ewm(span=signal, adjust=False).mean()
    return signal


features = []
for c in data.columns[2:]:
    s = Stream.source(list(data[c]), dtype="float").rename(data[c].name)
    features += [s]

cp = Stream.select(features, lambda s: s.name == "close")

features = [
    cp.log().diff().rename("lr"),
    rsi(cp, period=20).rename("rsi"),
    macd(cp, fast=10, slow=50, signal=5).rename("macd")
]

feed = DataFeed(features)

feed.compile()

In [4]:
for i in range(5):
    print(feed.next())

{'lr': nan, 'rsi': nan, 'macd': 0.0}
{'lr': -0.0021132537317512523, 'rsi': 0.0, 'macd': -0.00025383244206770666}
{'lr': 0.0005861432733618444, 'rsi': 22.585075537924805, 'macd': -0.00029659793912712514}
{'lr': 0.003896237122257923, 'rsi': 70.03888631580796, 'macd': 0.00017354293391302488}
{'lr': 0.0007254090599987661, 'rsi': 73.25880241509162, 'macd': 0.0004950867867173517}


## Setup Trading Environment

In [5]:
from tensortrade.oms.instruments import Instrument
from tensortrade.oms.orders import (
    Broker,
    Order,
    OrderListener,
    OrderSpec,
    proportion_order,
    risk_managed_order,
    TradeSide,
    TradeType
)

USD = Instrument("USD", 6, "U.S. Dollar")
CAD = Instrument("CAD", 6, "Canadian Dollar")

oanda = Exchange("oanda", service=execute_order)(
    Stream.source(list(data["close"]), dtype="float").rename("CAD-USD")
)

# cash = Wallet(oanda, 10000 * USD)
# asset = Wallet(oanda, 1000 * CAD)

cash = Wallet(oanda, 1000 * CAD)
asset = Wallet(oanda, 10000 * USD)

portfolio = Portfolio(CAD, [
    cash,
    asset
])


renderer_feed = DataFeed([
    Stream.source(list(data["date"])).rename("date"),
    Stream.source(list(data["open"]), dtype="float").rename("open"),
    Stream.source(list(data["high"]), dtype="float").rename("high"),
    Stream.source(list(data["low"]), dtype="float").rename("low"),
    Stream.source(list(data["close"]), dtype="float").rename("close"), 
    Stream.source(list(data["volume"]), dtype="float").rename("volume") 
])


env = default.create(
    portfolio=portfolio,
    # action_scheme="managed-risk",
    # action_scheme=default.actions.ManagedRiskOrders(),
    # action_scheme=default.actions.BSH(cash, asset),
    action_scheme=default.actions.SimpleOrders(
                 trade_type=TradeType.MARKET,
                 min_order_pct=0.5),
    reward_scheme=default.rewards.SimpleProfit(),
    # reward_scheme="risk-adjusted",
    feed=feed,
    renderer_feed=renderer_feed,
    # renderer=default.renderers.PlotlyTradingChart(),
    renderer="screen-log",
    window_size=20
)

env.observer.feed.next()

{'internal': {'oanda:/CAD-USD': 1.26479,
  'oanda:/CAD:/free': 1000.0,
  'oanda:/CAD:/locked': 0.0,
  'oanda:/CAD:/total': 1000.0,
  'oanda:/USD:/free': 10000.0,
  'oanda:/USD:/locked': 0.0,
  'oanda:/USD:/total': 10000.0,
  'oanda:/USD:/worth': 12647.900000000001,
  'net_worth': 13647.900000000001},
 'external': {'lr': nan, 'rsi': nan, 'macd': 0.0},
 'renderer': {'date': '2018-02-25 10:00 PM',
  'open': 1.26496,
  'high': 1.2663,
  'low': 1.26405,
  'close': 1.26479,
  'volume': 8890}}

## Setup and Train Agent

In [ ]:
# agent = DQNAgent(env)

# agent.train(n_steps=1000, n_episodes=20, save_path="agents_forex/")

In [8]:
# from stable_baselines.common.policies import MlpLnLstmPolicy
# from stable_baselines import PPO2

# model = PPO2
# policy = MlpLnLstmPolicy
# params = { "learning_rate": 1e-5 }

# agent = model(policy, environment, model_kwargs=params)
# agent.train(n_steps=1000, n_episodes=20, save_path="agents_forex/")

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [13]:
# from tensorforce.agents import Agent

# agent_spec = {
#     "type": "ppo_agent",
#     "step_optimizer": {
#         "type": "adam",
#         "learning_rate": 1e-4
#     },
#     "discount": 0.99,
#     "likelihood_ratio_clipping": 0.2,
# }

# network_spec = [
#     dict(type='dense', size=64, activation="tanh"),
#     dict(type='dense', size=32, activation="tanh")
# ]

# agent = Agent.create(spec=agent_spec,
#                         kwargs=dict(network=network_spec,
#                                     states=env.states,
#                                     actions=env.actions))

AttributeError: 'TradingEnv' object has no attribute 'states'

In [ ]:
# Run until episode ends
episode_reward = 0
done = False

# obs = env.reset()
# while not done:
#     action = agent.compute_action(obs)
#     obs, reward, done, info = env.step(action)
#     episode_reward += reward

state  = env.reset()
# while not done:
for _ in range(len(data.index)):
    action = agent.get_action(state, threshold=0)
    next_state, reward, done, info = env.step(action)
    print(info)
    episode_reward += reward
    state = next_state

env.render()

In [ ]:
performance = pd.DataFrame.from_dict(env.action_scheme.portfolio.performance, orient='index')

# copy performance data to clipboard for analysis using Number
# performance.to_clipboard(index=True)
performance.to_csv('USD_CAD_Performance.csv')
performance.plot()

from pandasgui import show
gui = show(performance)

In [ ]:
ledger = env.action_scheme.portfolio.ledger.as_frame()
# portfolio.ledger.as_frame().head(20)
gui = show(ledger)
# portfolio.ledger.as_frame().to_clipboard(index=False)